# TC 3006B Inteligencia artificial avanzada para la ciencia de datos I
# CIFAR-10 Data Preparation (PyTorch)
## Feedback

## Deep Learning -- Data Cleaning

### Objective: Load CIFAR-10, explore data, compute per-channel statistics (mean & std), normalize images, and verify normalization.

### Instructions
- Focus on data preparation, analysis, data is not usually as nice as CIFAR10 is. Model training follows as explained in classes.
- Write comments and reflections in the indicated, verify you answer questions in Markdown cells.
- Keep your code clean, reproducible, and well‑commented.



## Environment & Reproducibility


In [6]:
# Install libraries as needed, if using Colab, maybe all good

import os, json, random, math, time
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

##########################################
SEED = 42 #for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
###########################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu



## Download & Load CIFAR-10 (Train/Test Splits)


#### If using Colab you may need the following

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/7mo Semestre/')

In [7]:
# Maybe obvious, adjust path according to your needs
DATA_DIR = Path("/content/drive/MyDrive/Semestre7/cifar-10")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Base transform for statistics: convert to tensor only (no normalization/augmentations yet)
base_transform = transforms.ToTensor()

train_set = datasets.CIFAR10(root=DATA_DIR, train=True, download=True, transform=base_transform)
test_set  = datasets.CIFAR10(root=DATA_DIR, train=False, download=True, transform=base_transform)

len(train_set), len(test_set), train_set.data.shape


100%|██████████| 170M/170M [00:04<00:00, 37.6MB/s]


(50000, 10000, (50000, 32, 32, 3))


### Explore de dataset


In [8]:
classes = train_set.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [ ]:
def show_images(dataset, n=5):
    fig, axes = plt.subplots(1, n, figsize=(n*2.5, 2.5))
    for ax in axes:
        idx = np.random.randint(0, len(dataset))
        img, label = dataset[idx]
        # img: tensor [C,H,W] in [0,1]
        ax.imshow(np.transpose(img.numpy(), (1, 2, 0)))  # HWC
        ax.set_title(classes[label])
        ax.axis("off")
    plt.tight_layout()
    plt.show()

show_images(train_set, n=3)



> **Reflection (Markdown):** Describe CIFAR‑10 (resolution, channels, number of classes/samples). Include one sentence on why normalization is helpful for training.

- Resolution: 32×32 pixels
- Channels: 3 (RGB color images)
- Classes: 10 (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
- Samples: 50,000 training + 10,000 test images (6,000 per class)

Normalizing pixels values to have zero mean and unit variance, this helps training by keeping gradients stable and allowing us to use higher learning rates which would mean faster and more stable convergence


## Compute Per‑Channel Mean & Standard Deviation



In [10]:
BATCH_SIZE = 512
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

'''
Compute mean and std, before normalization

'''
# Make sure DataLoader has shuffle=False
n_channels = 3
psum = torch.zeros(n_channels)
psqsum = torch.zeros(n_channels)
npixels = 0

for x, _ in train_loader:
    # x: [B, C, H, W], values in [0,1] after ToTensor()
    b, c, h, w = x.shape
    psum    += x.sum(dim=[0, 2, 3])
    psqsum  += (x * x).sum(dim=[0, 2, 3])
    npixels += b * h * w

mean = psum / npixels
var  = (psqsum / npixels) - mean**2
std  = var.sqrt()

print("Mean:", mean)
print("Std:", std)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Mean: tensor([0.4914, 0.4822, 0.4465])
Std: tensor([0.2470, 0.2435, 0.2616])



> Record your computed values here (e.g., `Mean ≈ [0.4914, 0.4822, 0.4465]`, `Std ≈ [0.2470, 0.2435, 0.2616]`). Your numbers should closely match these canonical references.



## Normalize Datasets Using Computed Stats


In [11]:
normalize = transforms.Normalize(mean=mean.tolist(), std=std.tolist())

train_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

train_set_norm = datasets.CIFAR10(root=DATA_DIR, train=True, download=False, transform=train_transform)
test_set_norm  = datasets.CIFAR10(root=DATA_DIR, train=False, download=False, transform=test_transform)

len(train_set_norm), len(test_set_norm)


(50000, 10000)


## Verify Normalization (Means ≈ 0, Stds ≈ 1)

Confirm the transformation worked as intended.


In [13]:
train_loader_norm = DataLoader(train_set_norm, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

'''
Verify that data are normalized
'''
mean_norm = 0.0
std_norm = 0.0
nb_samples_norm = 0.
for data_norm, _ in train_loader_norm:
    batch_samples_norm = data_norm.size(0)
    data_norm = data_norm.view(batch_samples_norm, data_norm.size(1), -1)
    mean_norm += data_norm.mean(2).sum(0)
    std_norm += data_norm.std(2).sum(0)
    nb_samples_norm += batch_samples_norm

mean_norm /= nb_samples_norm
std_norm /= nb_samples_norm
print("Normalized Mean:", mean_norm)
print("Normalized Std:", std_norm)

Normalized Mean: tensor([ 6.2943e-08, -3.7561e-07, -3.9219e-07])
Normalized Std: tensor([0.8189, 0.8190, 0.7682])



> Report the post‑normalization means and stds. They should be close to `[0,0,0]` and `[1,1,1]` (small deviations are normal due to rounding/batching).



## Light Data Augmentation
Add a few basic augmentations (random crop with padding, random horizontal flip). Use **train** only.


In [14]:

aug_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    #anything else you may want to add is welcomed
    normalize,
])

train_set_aug = datasets.CIFAR10(root=DATA_DIR, train=True, download=False, transform=aug_transform)
len(train_set_aug)


50000


> Explain when/why augmentation helps. What risks do heavy augmentations pose for small images like CIFAR‑10 (32×32)?

When/why
Augmentation helps when we have small datasets or overfitting since it forces the model to learn invariant features

Risks
When using small images like CIFAR-10, augmentation tends to destryo essential details


## Class Distribution
Confirm class balance. CIFAR‑10 should be balanced across the 10 classes. Create a plot as a visual aid


In [ ]:
'''
Cifar10 is perfectly distributed among classes, this will not always be the case, plots help, numbers help
'''

# show class distribution
# show plot


## 9. Submission Checklist
- [ ] Printed dataset sizes and showed random sample images.
- [ ] Reported per‑channel mean & std from the training set.
- [ ] Applied normalization using computed stats.
- [ ] Verified post‑normalization mean/std are ~0/1.
- [ ] Basic augmentations and rationale.
